#### Name: Turki Alqahtani
#### ID: 2236938

# Import Library

In [1]:
import numpy as np
import os
from PIL import Image

# Set My Own Seed

In [2]:
# Set random seed for reproducible results
np.random.seed(42)

# Creating ANN Class with Needed Methods

In [3]:
class ANN:
    def __init__(self, input_size):
        # First hidden layer
        self.w1 = 0.01 * np.random.randn(input_size, 32)
        self.b1 = np.zeros((1, 32))
        # Second hidden layer
        self.w2 = 0.01 * np.random.randn(32, 16)
        self.b2 = np.zeros((1, 16))
        # Output layer
        self.w3 = 0.01 * np.random.randn(16, 10)
        self.b3 = np.zeros((1, 10))
    
    def forward(self, X):
        # First hidden layer
        z1 = np.dot(X, self.w1) + self.b1
        a1 = np.maximum(0, z1)  # ReLU
        # Second hidden layer
        z2 = np.dot(a1, self.w2) + self.b2
        a2 = np.maximum(0, z2)  # ReLU
        # Output layer
        z3 = np.dot(a2, self.w3) + self.b3
        exp_z = np.exp(z3 - np.max(z3, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)  # Softmax
    
    def loss(self, pred, y):
        return -np.mean(np.log(np.clip(pred[range(len(pred)), y], 1e-7, 1)))
    
    def save(self, name):
        np.savez(name, w1=self.w1, b1=self.b1, w2=self.w2, b2=self.b2, w3=self.w3, b3=self.b3)
    
    def load(self, name):
        data = np.load(name)
        self.w1, self.b1 = data['w1'], data['b1']
        self.w2, self.b2 = data['w2'], data['b2']
        self.w3, self.b3 = data['w3'], data['b3']

# Creating Data Preprocessing Class

In [4]:
def load_data(path):
    X, y = [], []
    for digit in range(10):
        folder = os.path.join(path, str(digit))
        if os.path.exists(folder):
            for file in os.listdir(folder):
                if file.endswith(('.png')):
                    img = Image.open(os.path.join(folder, file))
                    arr = np.array(img, dtype=np.float64)
                    flat_arr = arr.flatten()
                    if flat_arr.max() > 1: flat_arr /= 255.0
                    X.append(flat_arr)
                    y.append(digit)
    return np.array(X), np.array(y)

# **Data Preprocessing**

In [5]:
# Load data
X, y = load_data("Ai2_Dataset/Turki Q")
print(f"Loaded {len(X)} images")
print(f"Image shape: {X[0].shape}")
print(f"Classes: {np.unique(y)}")

Loaded 1980 images
Image shape: (784,)
Classes: [0 1 2 3 4 5 6 7 8 9]


# **Creating Model**

In [6]:
#Create model
model = ANN(X.shape[1])
print(f"Model created with input size: {X.shape[1]}")
print(f"Architecture: {X.shape[1]} -> 32 -> 16 -> 10")

Model created with input size: 784
Architecture: 784 -> 32 -> 16 -> 10


# **Model Training**

In [7]:
# Train
print("Starting training...")
best_loss = 999
for i in range(5000):
    # Random weight updates for all layers
    model.w1 += 0.01 * np.random.randn(*model.w1.shape)
    model.b1 += 0.01 * np.random.randn(*model.b1.shape)
    model.w2 += 0.01 * np.random.randn(*model.w2.shape)
    model.b2 += 0.01 * np.random.randn(*model.b2.shape)
    model.w3 += 0.01 * np.random.randn(*model.w3.shape)
    model.b3 += 0.01 * np.random.randn(*model.b3.shape)
    
    pred = model.forward(X)
    loss = model.loss(pred, y)
    
    if loss < best_loss:
        best_loss = loss
        model.save("Turki2236938Model.npz")
        acc = np.mean(np.argmax(pred, axis=1) == y)
        print(f"Step {i}: Loss={loss:.3f}, Acc={acc:.3f}")
    else:
        model.load("Turki2236938Model.npz")
print("Training completed!")

Starting training...
Step 0: Loss=2.303, Acc=0.099
Step 2: Loss=2.303, Acc=0.104
Step 4: Loss=2.303, Acc=0.100
Step 7: Loss=2.303, Acc=0.100
Step 10: Loss=2.303, Acc=0.088
Step 17: Loss=2.303, Acc=0.100
Step 19: Loss=2.302, Acc=0.100
Step 21: Loss=2.302, Acc=0.100
Step 26: Loss=2.302, Acc=0.100
Step 27: Loss=2.302, Acc=0.100
Step 30: Loss=2.302, Acc=0.100
Step 37: Loss=2.302, Acc=0.100
Step 40: Loss=2.302, Acc=0.100
Step 41: Loss=2.302, Acc=0.100
Step 42: Loss=2.301, Acc=0.100
Step 46: Loss=2.300, Acc=0.100
Step 48: Loss=2.300, Acc=0.192
Step 51: Loss=2.300, Acc=0.192
Step 52: Loss=2.300, Acc=0.178
Step 53: Loss=2.299, Acc=0.129
Step 57: Loss=2.299, Acc=0.211
Step 59: Loss=2.298, Acc=0.206
Step 62: Loss=2.298, Acc=0.177
Step 65: Loss=2.298, Acc=0.171
Step 70: Loss=2.297, Acc=0.179
Step 71: Loss=2.297, Acc=0.164
Step 72: Loss=2.296, Acc=0.178
Step 77: Loss=2.295, Acc=0.109
Step 79: Loss=2.295, Acc=0.096
Step 80: Loss=2.294, Acc=0.143
Step 81: Loss=2.294, Acc=0.100
Step 83: Loss=2.293, A

# **Loading Result Model**

In [8]:
model.load("Turki2236938Model.npz")

# **Test on shared test set**

In [9]:
test_path = "Ai2_Dataset/SharedTesting"
if os.path.exists(test_path):
    print("Loading shared test set...")
    test_X, test_y = load_data(test_path)
    test_pred = model.forward(test_X)
    test_acc = np.mean(np.argmax(test_pred, axis=1) == test_y)
    print(f"Shared Test Accuracy: {test_acc:.3f}")

Loading shared test set...
Shared Test Accuracy: 0.700


# **Print every prediction**

In [10]:
for i, pred in enumerate(test_pred):
    predicted_class = np.argmax(pred)
    actual_class = test_y[i]
    print(f"Sample {i}: Predicted={predicted_class}, Actual={actual_class}")
else:
    print("Done")

Sample 0: Predicted=3, Actual=0
Sample 1: Predicted=0, Actual=0
Sample 2: Predicted=0, Actual=0
Sample 3: Predicted=0, Actual=0
Sample 4: Predicted=9, Actual=0
Sample 5: Predicted=3, Actual=0
Sample 6: Predicted=0, Actual=0
Sample 7: Predicted=0, Actual=0
Sample 8: Predicted=0, Actual=0
Sample 9: Predicted=0, Actual=0
Sample 10: Predicted=1, Actual=1
Sample 11: Predicted=1, Actual=1
Sample 12: Predicted=1, Actual=1
Sample 13: Predicted=1, Actual=1
Sample 14: Predicted=4, Actual=1
Sample 15: Predicted=1, Actual=1
Sample 16: Predicted=1, Actual=1
Sample 17: Predicted=1, Actual=1
Sample 18: Predicted=4, Actual=1
Sample 19: Predicted=1, Actual=1
Sample 20: Predicted=2, Actual=2
Sample 21: Predicted=3, Actual=2
Sample 22: Predicted=2, Actual=2
Sample 23: Predicted=2, Actual=2
Sample 24: Predicted=2, Actual=2
Sample 25: Predicted=9, Actual=2
Sample 26: Predicted=2, Actual=2
Sample 27: Predicted=2, Actual=2
Sample 28: Predicted=2, Actual=2
Sample 29: Predicted=3, Actual=2
Sample 30: Predicted